In [38]:
import itertools
import pickle
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

rng = np.random.RandomState(42)

In [6]:
data = pd.read_csv("../data/train_data_features.csv").drop('Unnamed: 0', axis=1)
data.fillna(value="", inplace=True)

In [39]:
embedings = np.load('../w2v_embeddings/embedded_titles_100.npy')

X = data.drop(['fake_news_score',
              "click_bait_score",
              "Content Title",
              "Content Url",
              "Content Published Time",
              "Content"], axis=1).fillna(value=0).as_matrix()

y = data.fake_news_score.as_matrix()
y = (y - 1) / 2

embedings_train, embedings_test, X_train, X_test, y_train, y_test = train_test_split(embedings, X, y, random_state=rng)

In [40]:
embedings_train.shape

(2111, 41, 100)

In [41]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers.advanced_activations import ELU


model = Sequential()
model.add(Conv1D(512, 7, activation='relu', input_shape=(41, 100)))
model.add(Dropout(0.3))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(64, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_58 (Conv1D)           (None, 35, 512)           358912    
_________________________________________________________________
dropout_33 (Dropout)         (None, 35, 512)           0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 31, 128)           327808    
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 10, 128)           0         
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 8, 64)             24640     
_________________________________________________________________
global_average_pooling1d_14  (None, 64)                0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 64)                0         
__________

In [42]:
model.fit(embedings_train, y_train, batch_size=512, epochs=50, validation_split=0.1)
score = model.evaluate(embedings_test, y_test, batch_size=512)

Train on 1899 samples, validate on 212 samples
Epoch 1/50
1899/1899 [==============================] - 1s - loss: 0.6970 - acc: 0.6804 - val_loss: 0.6666 - val_acc: 0.6887
Epoch 2/50
1899/1899 [==============================] - 0s - loss: 0.6320 - acc: 0.6851 - val_loss: 0.6293 - val_acc: 0.6887
Epoch 3/50
1899/1899 [==============================] - 0s - loss: 0.6049 - acc: 0.6851 - val_loss: 0.6198 - val_acc: 0.6887

In [43]:
dict(zip(model.metrics_names, score))

{'acc': 0.69886364720084448, 'loss': 0.95372361486608337}

In [12]:
{'acc': 0.73295455629175355, 'loss': 1.4387078285217285}

{'acc': 0.73295455629175355, 'loss': 1.4387078285217285}

In [34]:
model = Sequential()
model.add(Conv1D(64, 5, activation='relu', input_shape=(41, 100)))
model.add(Dropout(0.3))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(512, 1, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_55 (Conv1D)           (None, 37, 64)            32064     
_________________________________________________________________
dropout_31 (Dropout)         (None, 37, 64)            0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 33, 128)           41088     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 11, 128)           0         
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 11, 512)           66048     
_________________________________________________________________
global_average_pooling1d_13  (None, 512)               0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 512)               0         
__________